# Conversational UI Chatbot App with ChatGPT, LangChain and Streamlit

Here we will build a advanced ChatGPT Conversational UI-based chatbot using LangChain and Streamlit with the following features:

- Custom Landing Page
- Conversational memory
- Result streaming capabilities (Real-time output)

## Install App and LLM dependencies

In [1]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
# !pip install pyngrok==7.1.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.10
    Uninstalling langchain-core-0.3.10:
      Successfully uninstalled langchain-core-0.3.10
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.0
    Uninstalling langchain-text-splitters-0.3.0:
      Successfully uninstalled langchain-text-splitters-0.3.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.3
    Uninstalling langchain-0.3.3:
      Successfully uninstalled langchain-0.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.2 requires langchain-core<0.4.0,>=0.3.9, but you have langchain-core 0.

## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [15]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [16]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

## Write the app code here and store it in a py file

In [18]:
%%writefile app.py
# that will write all the code below it to the python file app.py
# we will then deploy this app.py file on the cloud server where colab is running
# if you have your own server you can just write the code in app.py and deploy it directly


from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from operator import itemgetter
import streamlit as st

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

# Customize initial app landing page
st.set_page_config(page_title="AI Assistant", page_icon="🤖")
st.title("Welcome I am AI Assistant 🤖")

# Manages live updates to a Streamlit app's display by appending new text tokens
# to an existing text stream and rendering the updated text in Markdown
class StreamHandler(BaseCallbackHandler):
  def __init__(self, container, initial_text=""):
    self.container = container
    self.text = initial_text

  def on_llm_new_token(self, token: str, **kwargs) -> None:
    self.text += token
    self.container.markdown(self.text)

# Load a connection to ChatGPT LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1,
                     streaming=True)

# Add a basic system prompt for LLM behavior
SYS_PROMPT = """
              Act as a helpful assistant and answer questions to the best of your ability.
              Do not make up answers.
              """

# Create a prompt template for langchain to use history to answer user prompts
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
  ]
)

# Create a basic llm chain
llm_chain = (
  prompt
  |
  chatgpt
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory()

# Create a conversation chain
conversation_chain = RunnableWithMessageHistory(
  llm_chain,
  lambda session_id: streamlit_msg_history,  # Accesses memory
  input_messages_key="input",
  history_messages_key="history",
)

# Shows the first message when app starts
if len(streamlit_msg_history.messages) == 0:
  streamlit_msg_history.add_ai_message("How can I help you?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    # Initializing an empty data stream
    stream_handler = StreamHandler(st.empty())
    config = {"configurable": {"session_id": "any"},
              "callbacks": [stream_handler]}
    # Get llm response
    response = conversation_chain.invoke({"input": user_prompt},
                                         config)


Overwriting app.py


## Start the app

In [23]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

OSError: Background processes not supported.

In [ ]:
!python -m streamlit run app.py --server.port=8989 &>./logs.txt &

In [ ]:
'''
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)
'''

Streamlit App: https://0430-35-221-238-153.ngrok-free.app


## Remove running app processes

In [ ]:
ngrok.kill()

In [ ]:
!ps -ef | grep streamlit

root         637       1  1 03:14 ?        00:00:06 /usr/bin/python3 /usr/local/bin/streamlit run ap
root        2854     203  0 03:23 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        2856    2854  0 03:23 ?        00:00:00 grep streamlit


In [ ]:
!sudo kill -9 637